In [68]:
import argparse
import pickle as pkl
import pandas as pd
from pathlib import Path
from tqdm import tqdm
import matplotlib.pyplot as plt
# from clickstream_experiment.source.clickstream import ClickStream
from gensim.models import KeyedVectors
from gensim.models.word2vec import LineSentence
# from FlowHMM.model.discretized_HMM import DiscreteHMM

In [6]:
DATA_SET = 'train'
PROJECT_PATH = '/ziob/klaudia/Multiobjective_RS'
w2v_dim = 100


In [71]:
data_path = f"{PROJECT_PATH}/clickstream_experiment/data/preprocessed_data/train_valid_data_{w2v_dim}_{5}_{5}.pkl"
if Path(data_path).exists():
    with open(data_path, 'rb') as f:
        data = pkl.load(f)
print(data.keys())

dict_keys(['Xd_train', 'Xd_test', 'Xc_train', 'Xc_test', 'lengths_train', 'lengths_sub_train', 'lengths_test', 'myHMM.nodes'])


In [72]:
data['myHMM.nodes']

array([29, 57, 45, ...,  8, 69,  5])

In [9]:
vectors = KeyedVectors.load(f'{PROJECT_PATH}/data/preprocessed_data/vectors_train_100.kv')

In [14]:
vectors.word_vec()

/tmp/ipykernel_1968135/2997131487.py:1: DeprecationWarning: Call to deprecated `word_vec` (Use get_vector instead).
  vectors.word_vec()


TypeError: KeyedVectors.get_vector() missing 1 required positional argument: 'key'

In [21]:
vectors.key_to_index

AttributeError: The index2entity attribute has been replaced by index_to_key since Gensim 4.0.0.
See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4

In [28]:
import numpy as np
np.concatenate([vectors.get_vector(k).reshape(1, -1) for k in list(vectors.key_to_index.keys())])

array([[ 0.37812564,  0.9615767 ,  0.9456675 , ..., -1.8996654 ,
         1.8654928 , -0.6869288 ],
       [-0.0635553 ,  2.4719412 ,  0.12076852, ...,  0.5209727 ,
         0.2821499 , -0.03281005],
       [-0.12845026, -0.72366726, -2.040437  , ..., -0.00900516,
         2.5470822 ,  0.42293575],
       ...,
       [-0.03972329,  0.00678696, -0.02461287, ..., -0.02534718,
         0.08919474, -0.0304991 ],
       [-0.06562883,  0.05732198, -0.02860171, ..., -0.00453127,
         0.01211101, -0.10140423],
       [-0.02814574,  0.04212878, -0.04877787, ..., -0.01338863,
        -0.02220276, -0.04998498]], dtype=float32)